In [7]:
import os

In [8]:
%pwd

'C:\\Users\\peguy\\Documents\\end-to-end-machine-learning-project-with-mlflow\\research'

In [9]:
os.chdir('../')

In [10]:
%pwd


'C:\\Users\\peguy\\Documents\\end-to-end-machine-learning-project-with-mlflow'

In [11]:
from dataclasses import  dataclass
from pathlib import  Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [12]:
from mlproject.constants import  *
from mlproject.utils.common import  read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [14]:
import  os
import urllib.request as request
import zipfile
from mlproject import  logger
from mlproject.utils.common import  get_size

In [15]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise  e


[2026-02-19 09:03:14,851]: INFO: common: Yaml file: config\config.yaml loaded successfully
[2026-02-19 09:03:14,853]: INFO: common: Yaml file: params.yaml loaded successfully
[2026-02-19 09:03:14,855]: INFO: common: Yaml file: schema.yaml loaded successfully
[2026-02-19 09:03:14,856]: INFO: common: Created directory at: artifacts
[2026-02-19 09:03:14,857]: INFO: common: Created directory at: artifacts/data_ingestion
[2026-02-19 09:03:15,457]: INFO: 2393258610: artifacts\data_ingestion\data.zip download! with following info: 
 Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A30C:53726:F62122:111AE83:6996C3C2
Accept-Ranges: bytes
Date: Thu, 